In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import requests
from PIL import Image, ImageTk
import io
import tkinter as tk
from tkinter import simpledialog

In [2]:
# 建立 VGG16 預訓練模型，不包含頂層
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 凍結預訓練模型的卷積層
for layer in base_model.layers:
    layer.trainable = False

# 添加自定義頂層進行二分類
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

In [5]:
# 建立完整模型
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 模型摘要
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [6]:
# 訓練數據集設置
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_dataset = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_dataset = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.


In [7]:
# 訓練模型
model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset)

Epoch 1/10
 23/103 [=====>........................] - ETA: 45s - loss: 2.6491 - accuracy: 0.7073

c:\Users\tsai\Desktop\aiot_data_analysis\tensorflow_gpu_env\lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


103/103 [==============================] - 76s 695ms/step - loss: 0.7035 - accuracy: 0.8913 - val_loss: 0.0801 - val_accuracy: 0.9731
Epoch 2/10
103/103 [==============================] - 39s 380ms/step - loss: 0.0944 - accuracy: 0.9633 - val_loss: 0.0533 - val_accuracy: 0.9817
Epoch 3/10
103/103 [==============================] - 43s 413ms/step - loss: 0.0627 - accuracy: 0.9792 - val_loss: 0.0962 - val_accuracy: 0.9682
Epoch 4/10
103/103 [==============================] - 41s 399ms/step - loss: 0.0572 - accuracy: 0.9805 - val_loss: 0.0691 - val_accuracy: 0.9743
Epoch 5/10
103/103 [==============================] - 41s 396ms/step - loss: 0.0616 - accuracy: 0.9783 - val_loss: 0.0359 - val_accuracy: 0.9866
Epoch 6/10
103/103 [==============================] - 41s 400ms/step - loss: 0.0623 - accuracy: 0.9774 - val_loss: 0.0375 - val_accuracy: 0.9866
Epoch 7/10
103/103 [==============================] - 41s 398ms/step - loss: 0.0559 - accuracy: 0.9792 - val_loss: 0.0373 - val_accuracy: 0.9

In [44]:
# 定義函數從 URL 加載圖像並進行預處理
def load_image_from_url(url):
    response = requests.get(url)
    image = Image.open(io.BytesIO(response.content)).convert('RGB')
    image = image.resize((224, 224))
    return image

# 定義分類函數
def test_image(image_url, model, class_names):
    image = load_image_from_url(image_url)
    image_array = np.array(image) / 255.0  # 正規化
    image_array = np.expand_dims(image_array, axis=0)  # 增加批次維度
    predictions = model.predict(image_array)
    predicted_class = list(class_names.keys())[np.argmax(predictions)]
    return predicted_class, image  # 返回分類結果與PIL圖像

In [45]:
# GUI 應用程式設置
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classification with VGG16")
        self.root.geometry("600x600")

        self.label = tk.Label(root, text="Enter image URL:")
        self.label.pack(pady=10)

        self.url_entry = tk.Entry(root, width=50)
        self.url_entry.pack(pady=10)

        self.classify_button = tk.Button(root, text="Classify Image", command=self.classify_image)
        self.classify_button.pack(pady=10)

        self.result_label = tk.Label(root, text="", font=("Helvetica", 16))
        self.result_label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

    def classify_image(self):
        image_url = self.url_entry.get()
        predicted_class, display_image = test_image(image_url, model, train_dataset.class_indices)
        
        # 更新分類結果
        self.result_label.config(text=f"Predicted Class: {predicted_class}")
        
        # 顯示圖片
        display_image = ImageTk.PhotoImage(display_image)
        self.image_label.config(image=display_image)
        self.image_label.image = display_image  # 保存對象的參考

In [46]:
# 創建主視窗
root = tk.Tk()
app = App(root)
root.mainloop()